<img src="NotebookAddons/DEVELOP_logo1.jpg" width="250" align="right"/>
<font face="Century Gothic">
<font size="6"> <b>
   Inundation Mapping from Time Series of Dual-Pol SAR Data <br></font><br>
    <font size="4"> <b><br>
NASA Jet Propulsion Laboratory <br><br> DEVELOP Program Fall 2018 & Spring 2019 - Alaska Wetland Mapping</b></font><br>
This Jupyter Notebook implements the SAR Wetland Extent Exploration Tool (SWEET), a thresholding-based approach to perform semi-automated mapping of wetland inundation.  Required inputs are intensity images of VV+VH polarization bands from Sentinel-1 C-SAR imagery.  Loaded images should already be co-registered, subset, radiometrically corrected, and in a projected coordinate system.  The notebook outputs classifications of typical, minimum, and maximum inundation states along with multi-looked classifications for individual dates.  The inundation classes are: open water, inundated, and not inundated. <br><br>
Additional information on the software requirements, Python dependencies, usage instructions, and project directory structure can be found in the README file.

**Purpose:** <br>
Process Sentinel-1 VH/VV SAR data and produce maps showing wetland inundation extent in Alaska. Alaska is used as an example for this exercise. Application to other areas is possible. Note, however, that due to the limited penetration of Sentinel-1 C-band SAR data, application in densely vegetated areas may lead to an underestimation of inundation extent.<br>
<img style="padding: 7px" src="NotebookAddons/website_Image2.png" width="400" align="right"/>

**Process:**
- Read in data (assumes data is already subset and coregistered)
- Select dates to include in analysis based on VV, VH, and VV/VH plots for each date and average backscatter timeseries plots (can exclude images that appear to have calibration errors)
- Calculate multi-temporal averages for VV, VH, and VV/VH (Ia)
- Classify Ia with thresholds to find typical inundation state
- Perform multi-looking on individual dates and classify them to obtain classifications for each
date
- Determine maximum and minimum inundation state from classified individual dates
- Export classified products as GeoTIFFs for validation 
<br><br>

**To do:**
- Implement calibration correction using selected calibration sites
- (Optional) Visual calibration checks:<br>
<t><t> a) Individual scene/multi-temporal average ratio plots <br>
<t><t> b) Average brightness in circle of variable radius around selected pixel over time
    </font>


<font face="Century Gothic">
<b>Background:</b><br>
Synthetic Aperture Radar (SAR) polarimetric backscatter is a powerful datasource, with its ability to penetrate cloud cover and acquire imagery in low-light conditions.  Sentinel-1's C-band wavelength is particularly useful for wetland mapping, as the longer C-band wavelength can penetrate vegetation canopies to detect areas of inundated vegetation.<br>
<br>
Sentinel-1 has dual polarization, where it can transmit a signal in either horizontal (H) or vertical (V) polarizations and receive signals back both vertically and horizontally.  This notebook uses both vertically transmitted vertically recieved (VV) and vertically transmitted horizontally received (VH) intensity images and backscatter values.<br>
<br>
With SAR, pulses of microwave energy are emitted and received by the radar instrument's atenna.  When the pulses hit smooth surfaces such as open water, the microwave energy scatters and reflects away, resulting in a low return of backscatter to the radar.  Thus, flat surfaces like water appear as dark targets in the image.  In cases of inundated vegetation, the microwaves scatter off both the flat open water and the vertically emerging vegetation, resulting in a "double-bounce" reflection.  In these double-bounce reflections, a large portion of the transmitted waves are reflected back to the radar, resulting in brighter areas in the image.<br>
<br>
VV signals are of interest as they capture the increased brightness from the "double bounce" effect that commonly occurs with areas of inundated vegetation. This effect is even stronger in HH and at L-band wavelengths. Hence, future systems such as NISAR will improve the inundation mapping performance further by providing HH polarization data at L-band.<br>
<br>
To create an algorithm to identiy areas of inundation, we determined brightness threshold characteristics of inundated areas.  To identify shifts in inundation, we also looked at the changes over time in image brightness.
</font>

In [ ]:
# Check Python version:
import sys
pn = sys.version_info[0]

In [ ]:
%%capture
# Import packages:
# Packages for analysis:
import rasterio #pip install rasterio (version >=1.0.8, requires GDAL >=2.3.1)
%matplotlib inline
import matplotlib #pip install matplotlib
import matplotlib.pyplot as plt
import numpy as np #pip install numpy
import copy
from datetime import datetime
from glob import glob
import os, subprocess, sys
# Interactive widgets:
import ipywidgets as widgets #pip install ipywidgets (included with Jupyter)
from ipywidgets import Layout, VBox, Label, Checkbox, GridBox
if pn == 2:
    import cStringIO #needed for the image checkboxes
elif pn == 3:
    import io
    import base64
# Interactive time slider plots:
import plotly #pip install plotly (version >= 3.0)
import plotly.graph_objs as go
from ipywidgets import interactive, HBox, VBox
# Interactive time slider:
import pandas as pd #pip install pandas
## Interactive plots w/ pixel selection
from mpldatacursor import datacursor
# For exporting:
from PIL import Image

import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()

In [ ]:
###############################################################################
# All functions and classes and defined here in advance of their usage
# below.  The original idea was to have this cell either hidden or 
# immutable for user's of this Notebook.
#
#### Define functions and classes:
class DataFile: 
    ''' Data structure for extracting and storing relevant metadata about each
    data file. Also contains methods for reading and closing data files.  For 
    now, assume all dates are in format YYYYMMDD and all file names follow the
    same naming convention.
    
    Parameters:
    - filepath: Path to the SAR image file being read
    - convention: Tells the algorithm which naming convention to expect, and
                consequently, how to extract metadata from the filename
                Options: "new" - expects filename of the form:
                               S1B_IW_RT10_20170423T040959_G_gpn_VV_subset.tif
                         "old" - expects filename of the form:
                               s1a-iw-rtch-vv-20170612T162820_coreg_subset.tif 
    Attributes:
    - path: Path to the SAR image
    - dir: Directory containing the SAR image
    - name: Filename of SAR image
    - ext: File extension of the SAR image (e.g .tif, .img)
    - vtype: Polarization type: vv or vh
    - date: Date string as extracted from filename (in form of yyyymmdd)
    - datep: Date parsed into datetime format
    - datef: Date formatted into a presentable string for labeling
    - mean: Mean value of pixels in image
    - min: Minimum value of pixels in image
    - max: Maximum value of pixels in image
    
    Methods:
    - read_file: Reads in SAR image file
    - read_data: Reads in 2d array of raster values
    - extract_metadata: Extracts date and polarization type from filename
    - calculate_mean: Calculates mean pixel value
    - calculate_min: Calculates minimum pixel value
    - calculate_max: Calculates maximum pixel value
    - close: Close SAR image file to reduce memory usage
    - plot: Plots the image
    '''
    def __init__(self, filepath, convention="new"):
        self.path = filepath
        self.dir = os.path.dirname(os.path.abspath(filepath))
        self.name = filepath.split('/')[-1]
        self.ext = filepath.split('.')[-1]
        [self.vtype, self.date] = self.extract_metadata(self.name, convention)
        self.datep = datetime.strptime(self.date, '%Y%m%d')
        self.datef = self.datep.strftime("%B %d, %Y")
        self.mean = self.calculate_mean()
        self.min = self.calculate_min()
        self.max = self.calculate_max()
        
    def read_file(self):
        '''Read in geospatial raster using rasterio'''
        self.raw = rasterio.open(self.path)
        return(self.raw)
    
    def read_data(self):
        '''Read in 2d array of raster values'''
        raw = self.read_file()
        data = raw.read(1, out_shape=(1, int(raw.height), int(raw.width)))
        return(data)
    
    def extract_metadata(self, filename, convention):
        '''Extract metadata (vtype and date) from filename, assumes common 
        naming convention
        '''
        if convention == "new": #used for 
            if "VV" in filename: vtype = "vv"
            elif "VH" in filename: vtype = "vh"
            else:
                vtype = "Null"
                print("Could not detect vtype")
            date = filename.split('_')[3].split('T')[0]
        elif convention == "old":
            if "-vv-" in filename: vtype = "vv"
            elif "-vh-" in filename: vtype = "vh"
            else: 
                vtype = "Null"
                print("Could not detect vtype")
            date = filename.split('-')[-1].split('T')[0]
        return([vtype, date])
    
    def calculate_mean(self):
        '''Calculate mean pixel value'''
        mean = np.mean(self.read_data())
        self.close()
        return(mean)
    
    def calculate_min(self):
        '''Calculate minimum pixel value'''
        minval = np.min(self.read_data())
        self.close()
        return(minval)
    
    def calculate_max(self):
        '''Calculate maximum pixel value'''
        maxval = np.max(self.read_data())
        self.close()
        return(maxval)
    
    def close(self):
        '''Close SAR image file to reduce memory usage'''
        self.raw.close()
        
    def plot(self):
        '''Plot image data contained in object'''
        image_axis = plt.imshow(self.read_data(), cmap = plt.cm.gist_gray)
        self.close()
        return(image_axis)

###############################################################################
        
class DateSet:
    '''Data structure to hold original, multi-looked, and classified VV and VH 
    images and calculated VV/VH for each date.
    
    Parameters:
    - date: Date string in form of yyyymmdd
    - vv_df: VV DataFile corresponding to date
    - vh_df: VH DataFile corresponding to date

    Attributes:
    - vv_df: VV DataFile corresponding to date
    - vh_df: VH DataFile corresponding to date
    - date: Date string in form of yyyymmdd
    - datep: Date parsed into datetime format
    - datef: Date formatted into a presentable string for labeling
    - r_max: Maximum pixel value in VV/VH ratio
    - r_min: Minimum pixel value in VV/VH ratio
    - r_mean: Mean value of pixels in VV/VH ratio
    - class_3x3: Classification product generated from multi-looked VV, VH, 
        and VV/VH values (appended to DateSet object after it is calculated)

    Methods:
    - close: Closes VV and VH image files
    '''
    def __init__(self, date, vv_df, vh_df):
        if vv_df.date == vh_df.date:
            self.vv_df = vv_df
            self.vh_df = vh_df
            self.date = date
            self.datep = vv_df.datep
            self.datef = vh_df.datef
            r = self.vv_df.read_data()/self.vh_df.read_data()
            self.r_min = np.min(r)
            self.r_max = np.max(r)
            self.r_mean = np.mean(r)
            self.close()
            self.class_3x3 = None
        else:
            print("Could not create DateSet object. VV and VH dates do not match")
    def close(self):
        '''Close VV and VH image files by calling close method in VV and VH
        DataFile objects
        '''
        self.vv_df.close()
        self.vh_df.close()
    
###############################################################################

def gray_plot(image, vmin=0, vmax=2, return_ax=False):
    '''Plots an image in grayscale.
    
    Parameters:
    - image: 2D array of raster values
    - vmin: Minimum value for colormap
    - vmax: Maximum value for colormap
    - return_ax: Option to return plot axis

    '''
    ax = plt.imshow(image, cmap = plt.cm.gist_gray)
    plt.clim(vmin,vmax)
    if return_ax:
        return(ax)

###############################################################################

def pixel2pixel_plot(image, vmin=0, vmax=.2, dpi=192, cursor=True, full=True):
    '''Plots a full resolution image of a map where each pixel in the map
    corresponds to a pixel in the user's monitor.
    
    Parameters:
    - image: 2d array of raster values
    - vmin: Minimum value for colormap
    - vmax: Maximum value for colormap
    - dpi: User monitor DPI
    - cursor: Option to include an interactive datacursor
    - full: Option to increase the width of the Jupyter Notebook to the full
        size of the user's monitor.
    '''
    if full:
        from IPython.core.display import display, HTML
        display(HTML("<style>.container { width:100% !important; }</style>"))
    xsize = np.size(image,1)
    ysize = np.size(image,0)
    plt.figure(figsize=(xsize/dpi, ysize/dpi),dpi=dpi)
    ax = gray_plot(image, vmin, vmax, True)
    if cursor:
        dc = datacursor(ax)
    plt.show()

    
###############################################################################

    
def big_fig(x=20,y=10):
    '''Initializes a large figure.
    
    Parameters:
    - x, y: X and Y figure dimensions
    '''
    return(plt.figure(figsize=(x,y)))

###############################################################################

def get_files(data_directory, ext="tif", search="", convention="new"):
    '''Returns a list of the files in the data directory that have the desired
    extension.  
    
    Parameters:
    - data_directory: Directory containing data files
    - ext: Desired file extension (default is .tif)
    - search: String that the function searchs for in filenames (default is "")
    - convention: Naming convention type expected by DataFile ("old" or "new")
    '''
    file_list = []
    ext_len = len(ext)
    for filename in os.listdir(data_directory):
        if filename[-ext_len::] == ext:
            if search not in filename: continue
            filepath = data_directory + filename
            data_file = DataFile(filepath, convention)
            file_list.append(data_file)
    return(file_list)

###############################################################################

def find_minmax(date_sets):
    '''Returns a list of nested lists containing minimum and maximum VV, VH, 
    and VV/VH values found from a list of DateSets to standardize color 
    limits when plotting.
    
    Parameters:
    - date_sets: List of DateSets to compare
    '''
    vv_running_min = 10
    vv_running_max = 0
    vh_running_min = 10
    vh_running_max = 0
    r_running_min = 10
    r_running_max = 0
    for ds in date_sets:
        # Check VV values
        if ds.vv_df.min < vv_running_min: vv_running_min = ds.vv_df.min
        if ds.vv_df.max > vv_running_max: vv_running_max = ds.vv_df.max
        # Check VH values
        if ds.vh_df.min < vh_running_min: vh_running_min = ds.vh_df.min
        if ds.vh_df.max > vh_running_max: vh_running_max = ds.vh_df.max
        # Check VV/VH values
        if ds.r_min < r_running_min: r_running_min = ds.r_min
        if ds.r_max > r_running_max: r_running_max = ds.r_max
    print("VV value range:")
    print(vv_running_min, vv_running_max)
    print("VH value range:")
    print(vh_running_min, vh_running_max)
    print("VV/VH value range:")
    print(r_running_min, r_running_max)
    return([[vv_running_min, vv_running_max], 
            [vh_running_min, vh_running_max], 
            [r_running_min, r_running_max]
           ]
          )

###############################################################################

def find_dates(data_files):
    '''Returns a list of unique dates in a list of VV and VH DataFiles.
    
    Parameters:
    - data_files: List of DataFiles to compare
    '''
    date_set = set()
    for df in data_files:
        date_set.add(df.date)
    date_list = np.sort(list(date_set)) #convert set to list to access by index
    return(date_list)
###############################################################################

def make_datesets(data_files, date_list):
    '''Returns a list of DateSet objects from list of DataFile objects and
    corresponding dates.
    
    Parameters:
    - data_files: List of DataFiles to combine into DateSets
    - date_list: List of corresponding dates
    '''
    date_dic = {}
    dateset_list = []
    for date in date_list:
        vtype_dic = {}
        date_dic[date] = vtype_dic
    for df in data_files:
        date = df.date
        vtype = df.vtype
        date_dic[date][vtype] = df
    for date in date_list:
        vtype_dic = date_dic[date]
        dateset = DateSet(date, vtype_dic['vv'], vtype_dic['vh'])
        dateset_list.append(dateset)
    return(dateset_list)

###############################################################################

def vv_vh_r_plot(vv, vh, r, 
                 vv_min, vv_max, 
                 vh_min, vh_max, 
                 r_min=0, r_max=6, 
                 axstr="",
                 xsize=9.5, ysize=3
                ):
    ''' Plots a 3 panel figure of VV, VH, and VV/VH for a given date.
    
    Parameters:
    - vv, vh, r: 2D arrays of VV, VH, and R values
    - vv_min, vv_max: Minimum and maximum VV values for colormap limits
    - vh_min, vh_max: Minimum and maximum VH values for colormap limits
    - r_min, r_max: Minimum and maximum VV/VH values for colormap limits
    - axstr: String to plot along right side of figure (default is "")
    - xsize, ysize: X and Y dimensions for figure
    '''
    fig = big_fig(xsize, ysize)
    # VV plot
    ax = plt.subplot(1, 3, 1)
    gray_plot(vv, vv_min, vv_max)
    plt.title("VV")
    # VH plot
    ax = plt.subplot(1, 3, 2)
    gray_plot(vh, vh_min, vh_max)
    plt.title("VH")
    # VV/VH plot
    ax = plt.subplot(1, 3, 3)
    gray_plot(r, r_min, r_max)
    plt.title("VV/VH")
    if axstr:
        # Set axis string
        ax.text(1.05, 0.5, axstr, rotation=90, ha='left',
                va='center', transform=ax.transAxes
               )
    return(fig)
###############################################################################

def vv_vh_cov_plot(vv_cov, vh_cov, 
                 vv_min, vv_max, 
                 vh_min, vh_max, 
                 axstr="",
                 xsize=9.5, ysize=3
                ):
    ''' Plots a 3 panel figure of VV, VH, and VV/VH for a given date.
    
    Parameters:
    - vv, vh, r: 2D arrays of VV, VH, and R values
    - vv_min, vv_max: Minimum and maximum VV values for colormap limits
    - vh_min, vh_max: Minimum and maximum VH values for colormap limits
    - r_min, r_max: Minimum and maximum VV/VH values for colormap limits
    - axstr: String to plot along right side of figure (default is "")
    - xsize, ysize: X and Y dimensions for figure
    '''
    fig = big_fig(xsize, ysize)
    # VV cov plot
    ax = plt.subplot(1, 2, 1)
    gray_plot(vv_cov, vv_min, vv_max)
    plt.title("VV Coefficient of Variation")
    # VH cov plot
    ax = plt.subplot(1, 2, 2)
    gray_plot(vh_cov, vh_min, vh_max)
    plt.title("VH Coefficient of Variation")
    if axstr:
        # Set axis string
        ax.text(1.05, 0.5, axstr, rotation=90, ha='left',
                va='center', transform=ax.transAxes
               )
    return(fig)

###############################################################################

def save_plot3(date_set, 
               vv_min, vv_max, 
               vh_min, vh_max, 
               r_min=0, r_max=6,
               pn=pn
              ):
    '''Returns a 3 panel (VV, VH, and VV/VH) plot saved using cStringIO so it
    can be used in multi_checkbox_widget_dateimages.
    
    Parameters:
    - date_set: DateSet object
    - vv_min, vv_max: Minimum and maximum VV values for colormap limits
    - vh_min, vh_max: Minimum and maximum VH values for colormap limits
    - r_min, r_max: Minimum and maximum VV/VH values for colormap limits
    - pn: Python version number (2 or 3)
    '''
    plt.ioff()
    vv = date_set.vv_df.read_data()
    vh = date_set.vh_df.read_data()
    r = vv/vh
    axstr = date_set.datef
    fig = vv_vh_r_plot(vv, vh, r, 
                       vv_min, vv_max, 
                       vh_min, vh_max, 
                       r_min, r_max, 
                       axstr
                      )
    date_set.close()
    if pn == 2:
        figdata = cStringIO.StringIO()
        fig.savefig(figdata, format='png')
        plt.close(fig)
        plt.ion()
        return(figdata)
    elif pn == 3:
        figdata = io.BytesIO()
        fig.savefig(figdata, format='png')
        plt.close(fig)
        plt.ion()
        return(base64.b64encode(figdata.getvalue()).decode())

###############################################################################

def multi_checkbox_widget_dateimages(date_sets, 
                                     vv_min, vv_max, 
                                     vh_min, vh_max, 
                                     r_min=0, r_max=6,
                                     pn=pn
                                    ):
    '''Returns a table of 3-paneled (VV, VH, and VV/VH) plots for each date
    with checkboxes next to each row so user can select dates to exclude based
    on visual inspection.
    
    Parameters:
    - date_sets: List of DateSet objects
    - vv_min, vv_max: Minimum and maximum VV values for colormap limits
    - vh_min, vh_max: Minimum and maximum VH values for colormap limits
    - r_min, r_max: Minimum and maximum VV/VH values for colormap limits
    - pn: Python version number (2 or 3)
    '''
    nds = len(date_sets)
    descriptions = [save_plot3(ds, vv_min, vv_max, vh_min, vh_max, r_min, r_max) 
                    for ds in date_sets
                   ]
    item_layout = Layout(height='auto', min_width='1000px')
    if pn == 2:
        items = [Checkbox(layout=item_layout, 
                          description='<img src=\"data:/png;base64,%s\"/>' % desc
                          .getvalue()
                          .encode("base64")
                          .strip(), 
                          value=False
                         ) for desc in descriptions
                ]
    elif pn == 3:
        items = [Checkbox(layout=item_layout,
                         description='<img src=\"data:/png;base64,%s\"/>' % desc
                         .strip(),
                         value=False
                         ) for desc in descriptions
                ]
    box_layout = Layout(border='3px solid black',
                        width='auto',
                        height='',
                       grid_template_columns='1',
                       grid_template_rows='auto'
                       )
    grid = GridBox(children=items, layout=box_layout)
    ws = VBox([Label('Select images to exclude:'), grid])
    return(ws)

###############################################################################
def get_excluded_dates3(ws, date_sets):
    '''Returns a list of dates that were selected by the user for exclusion
    in the image checkbox table.
    
    Parameters:
    - ws: Handle of image checkbox with selections
    - date_sets: List of DateSets used as input to the image checkbox 
    '''
    excluded_idx = [idx 
                    for idx, ws 
                    in enumerate(ws.children[1].children) 
                    if ws.value
                   ]
    excluded_dates = [date_sets[idx].datef for idx in excluded_idx]
    return(excluded_dates)

###############################################################################

def interactive_backscatter_plot(date_sets):
    '''Creates an interactive timeseries plot of backscatter values for
    a site.  Modified code from https://plot.ly/python/range-slider/
    
    Parameters:
    - date_sets: List of DateSets
    '''
    dateps = []
    vv_means = []
    vh_means = []
    for ds in date_sets:
        dateps.append(ds.datep)
        vv_means.append(10*np.log10(ds.vv_df.mean))
        vh_means.append(10*np.log10(ds.vh_df.mean))

    vv_trace = go.Scatter(
      x = dateps,
      y = vv_means,
      name = "vv",
      text = [str(vvm) for vvm in vv_means],
      yaxis = "y"
    )

    vh_trace = go.Scatter(
      x = dateps,
      y = vh_means,
      name = "vh",
      text = [str(vhm) for vhm in vh_means],
      yaxis = "y"
    )

    data = [vv_trace, vh_trace]

    # style all the traces
    for k in range(len(data)):
        data[k].update(
            {
                "hoverinfo": "name+x+text",
                "line": {"width": 0.5}, 
                "marker": {"size": 8},
                "mode": "lines+markers",
                "showlegend": False
            }
        )

    layout = {
      "dragmode": "zoom", 
      "hovermode": "x", 
      "legend": {"traceorder": "reversed"}, 
      "margin": {
        "t": 100, 
        "b": 100
      }, 
      "xaxis": {
        "autorange": True, 
        "range": ["2017-10-31 18:36:37.3129", "2018-05-10 05:23:22.6871"], 
        "rangeslider": {
          "autorange": True, 
          "range": ["2017-10-31 18:36:37.3129", "2018-05-10 05:23:22.6871"]
        }, 
        "type": "date"
      }, 
      "yaxis": {
        "anchor": "x", 
        "autorange": True, 
        "domain": [0, 1], 
        "linecolor": "#673ab7", 
        "mirror": True, 
        "range": [-20,20], 
        "showline": True, 
        "side": "right", 
        "tickfont": {"color": "#673ab7"}, 
        "tickmode": "auto", 
        "ticks": "",
        "title": "Power (dB)",
        "titlefont": {"color": "#673ab7"}, 
        "type": "linear", 
        "zeroline": False
      }
    }
    plotly.offline.init_notebook_mode(connected=True)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)
    
###############################################################################

def make_timeslider(date_sets):
    '''Creates and returns the handle to an interactive time slider
    that the user can use to select a time range of values to include.
    
    Parameters: 
    - date_sets: List of DateSets
    '''
    start_date = date_sets[0].datef
    end_date = date_sets[-1].datef
    date_range = pd.date_range(start_date, end_date, freq='D')
    options = [(date.strftime(' %b %d, %Y '), date) for date in date_range]
    index = (0, len(options)-1)
    selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'})
    return(selection_range_slider)  

###############################################################################

def get_slider_vals(selection_range_slider):
    '''Returns the minimum and maximum dates retrieved from the
    interactive time slider.
    
    Parameters:
    - selection_range_slider: Handle of the interactive time slider
    '''
    [a,b] = list(selection_range_slider.value)
    slider_min = a.to_pydatetime()
    slider_max = b.to_pydatetime()
    return(slider_min, slider_max)

###############################################################################

def filter_date_sets(date_sets, excluded_dates, slider_min, slider_max):
    '''Returns a list of DateSets that are filtered according to the
    excluded dates from the checkbox table and start/end dates retrieved from
    the interactive time slider.
    
    Parameters:
    - date_sets: List of DateSets to be filtered
    - excluded_dates: Dates chosen for exclusion from checkbox table
    - slider_min, slider_max: Minimum and maximum date values from time slider
    '''
    filtered_date_sets = []
    for ds in date_sets:
        in_time_range = ds.datep >= slider_min and ds.datep <= slider_max
        if ds.datef not in excluded_dates and in_time_range:
            filtered_date_sets.append(ds)
    return(filtered_date_sets)

###############################################################################

def plot_all_dates(date_sets, 
                   vv_min, vv_max, 
                   vh_min, vh_max, 
                   r_min=0, r_max=6, 
                   xsize=10, ysize=20):
    '''Plots  3-panel images (VV, VH, and VV/VH) for each date.
    
    Parameters:
    - date_sets: List of DateSets
    - vv_min, vv_max: Minimum and maximum VV values for colormap limits
    - vh_min, vh_max: Minimum and maximum VH values for colormap limits
    - r_min, vr_max: Minimum and maximum VV/VH values for colormap limits
    - xsize, ysize: X and Y figure dimensions
    '''
    big_fig(xsize,ysize) # initialize a large figure
    nds = len(date_sets)
    for ii in range(0, nds):
        pos = 3*ii + 1
        # VV plot
        ax = plt.subplot(nds, 3, pos)
        gray_plot(date_sets[ii].vv_df.read_data(), vv_min, vv_max)
        if ii == 0: plt.title("VV")
        # VH plot
        ax = plt.subplot(nds, 3, pos+1)
        gray_plot(date_sets[ii].vh_df.read_data(), vh_min, vh_max)
        if ii == 0: plt.title("VH")
        # VV/VH plot
        ax = plt.subplot(nds, 3, pos+2)
        gray_plot(
            date_sets[ii].vv_df.read_data()/date_sets[ii].vh_df.read_data(), 
            r_min, r_max
                 )
        ax.text(1.05, 0.5, date_sets[ii].datef, 
                rotation=90, 
                ha='left', 
                va='center', 
                transform=ax.transAxes
               )
        if ii == 0: plt.title("VV/VH")
        date_sets[ii].close()
        
###############################################################################

def calculate_temporal_avg(data_file_list, 
                           vtype="vh", 
                           start_date=0, 
                           end_date=9*10**13, 
                           verbose=False
                          ):
    '''Calculates and returns the multi-temporal average of VV or VH data 
    using a running sum.
    
    Parameters:
    - data_file_list: List of DataFiles.
    - vtype: SAR polarization type (vv or vh)
    - start_date, end_date: Start and end dates as numeric values
    - verbose: Option to print a message each time a DataFile is not of the
        correct vtype, or its date does not fall in the time range between
        start_date and end_date
    '''
    files_in_range = []
    count = 0
    for data_file in data_file_list:
        check = True
        while check:
            if data_file.vtype is not vtype:
                if verbose: print(
                    "Data is not of the correct polarization(VV or VH)"
                                 )
                break
            file_date = int(data_file.date)
            if file_date < start_date or file_date > end_date:
                if verbose: print(
                    "Data is not within the specified time frame"
                )
                break
            data = data_file.read_data()
            if count == 0:
                running_sum = data
                running_sum2 = data**2
            else:
                if data.shape != data.shape:
                    if verbose: print(
                        "Data are not the same dimensions (row x colum)"
                    )
                    break
                running_sum += data
                running_sum2 += data**2
            count += 1
            data_file.close()
            files_in_range.append(data_file)
            check = False
    temporal_mean = running_sum/count
    st_dev = np.sqrt((running_sum2 - (running_sum**2)/count)/(count-1))
    coef_of_variation = st_dev/temporal_mean
    return([temporal_mean, files_in_range, coef_of_variation])

###############################################################################

def calculate_r_cov(date_sets):
    '''Calculate the coefficient of variation for the VV/VH ratio.
    
    Parameters:
    - date_sets: List of DateSets
    '''
    count = 0
    for date in date_sets:
        vv = date.vv_df.read_data()
        vh = date.vh_df.read_data()
        r = vv/vh
        date.close()
        if count == 0:
            running_sum = r
            running_sum2 = r**2
        else:
            running_sum += r
            running_sum2 += r**2
        count += 1
    temporal_mean = running_sum/count
    st_dev = np.sqrt((running_sum2 - (running_sum**2)/count)/(count-1))
    r_cov = st_dev/temporal_mean
    return(r_cov)

###############################################################################

def calculate_multitemporal_avg(date_sets):
    '''Calculates and returns the multi-temporal VV, VH, and VV/VH averages
    from a list of DateSets.
    
    Parameters:
    - date_sets: List of DateSets
    '''
    vv_data_files = [ds.vv_df for ds in date_sets]
    vh_data_files = [ds.vh_df for ds in date_sets]
    [vv_avg, vv_files, vv_cov] = calculate_temporal_avg(vv_data_files, "vv")
    [vh_avg, vh_files, vh_cov] = calculate_temporal_avg(vh_data_files, "vh")
    r_avg = vv_avg/vh_avg
    return(vv_avg, vh_avg, r_avg, vv_cov, vh_cov)

###############################################################################

class ClassThresholds:
    '''Class that contains upper and lower bounds for VV, VH, and VV/VH for
    a class as well as a method for checking membership based on VV, VH, and 
    VV/VH values.
    
    Parameters:
    - vv_low, vv_hi: Lower and upper bounds for VV values in the class
    - vh_low, vh_hi: Lower and upper bounds for VH values in the class
    - r_low, r_hi: Lower and upper bounds for VV/VH values in the class
    
    Methods:
    - is_member: Check a set of VV, VH, and VV/VH values from a pixel to see
        if that pixel should be included in this class
    '''
    def __init__(self, 
                 vv_low=-9999, vv_hi=9999, 
                 vh_low=-9999, vh_hi=9999, 
                 r_low=-9999, r_hi=9999):
        self.vv_low = vv_low
        self.vv_hi = vv_hi
        self.vh_low = vh_low
        self.vh_hi = vh_hi
        self.r_low = r_low
        self.r_hi = r_hi
    
    def is_member(self, vv_val, vh_val, r_val):
        '''Checks if the input VV, VH, and VV/VH values are sufficient
        to classify the pixel in this class.
        '''
        vv_cond = vv_val > self.vv_low and vv_val < self.vv_hi
        vh_cond = vh_val > self.vh_low and vh_val < self.vh_hi
        r_cond = r_val > self.r_low and r_val < self.r_hi
        result = vv_cond and vh_cond and r_cond
        return(result)
    
###############################################################################

def pixel_radius_average(image, mid_row, mid_col, radius):
    '''Calculates and returns the average pixel value for an image in a 
    variable radius around a selected pixel.
    
    Parameters:
    - image: 2D array of raster values
    - mid_row, mid_col: Row and column coordinates of the selected pixel
    - radius: Radius size of the moving window (in pixels)
    '''
    # Find indices of pixels within the radius
    row_rng = range(mid_row-radius,mid_row+radius+1)
    col_rng = range(mid_col-radius,mid_col+radius+1)
    good_inds = []
    for row in row_rng:
        row_ind = row
        y_val = row - mid_row
        for col in col_rng:
            col_ind = col
            x_val = col - mid_col
            dist = x_val^2 + y_val^2
            if dist <= radius^2:
                good_inds.append([row, col])
    # Get values of pixels in radius
    vals_in_radius = []
    for pix in good_inds:
        row = pix[0]
        col = pix[1]
        try:
            val = image[row][col]
            vals_in_radius.append(val)
        except Exception as e:
            print("Could not add pixel to list; check edge effects")
            print(e)
    # Calculate average
    mean = np.mean(vals_in_radius)
    return(mean)

###############################################################################

def moving_window(data, radius=3, verbose=False):
    '''Applies a moving window average to a 2D array and returns the result.
    
    Parameters:
    - data: 2D array of raster values
    - radius: Radius size in pixels
    - verbose: Option to print a message whenever a pixel cannot be added to
        the window
    '''
    [nrow, ncol] = data.shape
    windowed = copy.copy(data)
    for row in range(nrow):
        for col in range(ncol):
            vals_in_window = []
            row_rng = range(row-radius,row+radius+1)
            col_rng = range(col-radius,col+radius+1)
            for y in row_rng:
                for x in col_rng:
                    try:
                        val = data[y][x]
                        vals_in_window.append(val)
                    except Exception as e:
                        if verbose:
                            print("Could not add pixel to list, " +
                                  "may be due to edge effects")
                            print(e)
            windowed[row][col] = np.mean(vals_in_window)
    return(windowed)

###############################################################################

def triple_classify(vv_data, vh_data, r_data,
                    water_thresh, flooded_thresh
                   ):
    '''Makes and returns a classification product based on VV, VH, and VV/VH
    values.
    
    Parameters:
    - vv_data, vh_data, r_data: VV, VH, and VV/VH arrays
    - water_thresh: ClassThreshold for the open water class
    - flooded_thresh: ClassThreshold for the inundated class
    '''
    classified_image = copy.copy(r_data)
    for row in range(classified_image.shape[0]):
        for col in range(classified_image.shape[1]):
            vv_val = float(vv_data[row][col])
            vh_val = float(vh_data[row][col])
            r_val = float(r_data[row][col])
            water = water_thresh.is_member(vv_val, vh_val, r_val)
            flooded = flooded_thresh.is_member(vv_val, vh_val, r_val)
            if water:
                classified_image[row][col] = 1
            elif flooded:
                classified_image[row][col] = 3
            else:
                classified_image[row][col] = 2
    return(classified_image)

###############################################################################

def plot_class_cov(typical_inundation, vv_cov, vh_cov,
                  xsize=10,ysize=5): 
    '''Makes and returns a classification product based on VV, VH, and VV/VH
    values.
    
    Parameters:
    - typical_inundation: 
    - vv_cov, vh_cov: VV and VH coefficient of variance (cov) distributions\
    - xsize, ysize: size parameters for the figure
    '''
    kwargs = dict(histtype='stepfilled', alpha=.8, bins=1000)
    nrows = 2
    ncols = 3
    npanels = nrows*ncols
    big_fig(xsize, ysize)
    for x in range(1,npanels+1):
        if x < 4:
            vtype = "VV"
            vals = vv_cov
            class_num = x
        else:
            vtype = "VH"
            vals = vh_cov
            class_num = x - 3
        indx = np.where(typical_inundation == class_num)
        ax = plt.subplot(nrows,ncols,x)
        plt.hist(vals[indx], **kwargs)
        ax.get_yaxis().set_visible(False)
        if x < 4:
            plt.title("Class: " + str(x))
        if x == 3:
            ax.text(1.05, 0.5, "VV Coefficient of Variation", rotation=90, ha='left',
                va='center', transform=ax.transAxes
               )
        if x == 6:
            ax.text(1.05, 0.5, "VH Coefficient of Variation", rotation=90, ha='left',
                va='center', transform=ax.transAxes
               )
        
###############################################################################

def refined_classify(vv_data, vh_data, r_data, vv_cov, vh_cov,
                    water_thresh, flooded_thresh, not_flooded_thresh,
                    nf_cov_thresh, f_cov_thresh, w_cov_thresh
                   ):
    '''Makes and returns a refined classification product based on the typical
    inundation state and VV coefficient of variation.
    
    Parameters:
    - vv_data, vh_data, r_data: VV, VH, and VV/VH arrays
    - vv_cov, vh_cov: VV and VH coefficient of variance (cov) distributions
    - water_thresh: ClassThreshold for the open water class
    - flooded_thresh: ClassThreshold for the inundated class
    - not_flooded_thresh: ClassThreshold for the not inundated class
    - nf_cov_thresh: cov threshold for temporal change in not inundated areas
    - f_cov_thresh: cov threshold for temporal change in inundated areas
    - w_cov_thresh: cov threshold for temporal change in open water areas
    '''
    classified_image = copy.copy(r_data)
    for row in range(classified_image.shape[0]):
        for col in range(classified_image.shape[1]):
            vv_val = float(vv_data[row][col])
            vh_val = float(vh_data[row][col])
            r_val = float(r_data[row][col])
            water = water_thresh.is_member(vv_val, vh_val, r_val)
            flooded = flooded_thresh.is_member(vv_val, vh_val, r_val)
            not_flooded = not_flooded_thresh.is_member(vv_val, vh_val, r_val)
            if water:
                if vv_cov[row][col] > w_cov_thresh:
                    classified_image[row][col] = 1
                else:
                    classified_image[row][col] = 0
            elif flooded:
                if vv_cov[row][col] > f_cov_thresh:
                    classified_image[row][col] = 4
                else:
                    classified_image[row][col] = 3
            else:
                if vv_cov[row][col] > nf_cov_thresh:
                    classified_image[row][col] = 4
                else:
                    classified_image[row][col] = 2
    return(classified_image)

###############################################################################

def refined_classify2(vv_data, vh_data, r_data, vv_cov, vh_cov,
                    water_thresh, flooded_thresh, not_flooded_thresh,
                    nf_vv_cov_thresh, f_vv_cov_thresh, w_vv_cov_thresh,
                    nf_vh_cov_thresh, f_vh_cov_thresh, w_vh_cov_thresh
                   ):
    '''Makes and returns a refined classification product based on the typical
    inundation state and VV & VH coefficients of variation.
    
    Parameters:
    - vv_data, vh_data, r_data: VV, VH, and VV/VH arrays
    - vv_cov, vh_cov: VV and VH coefficient of variance (cov) distributions
    - water_thresh: ClassThreshold for the open water class
    - flooded_thresh: ClassThreshold for the inundated class
    - not_flooded_thresh: ClassThreshold for the not inundated class
    - nf_cov_thresh: cov threshold for temporal change in not inundated areas
    - f_cov_thresh: cov threshold for temporal change in inundated areas
    - w_cov_thresh: cov threshold for temporal change in open water areas
    '''
    classified_image = copy.copy(r_data)
    for row in range(classified_image.shape[0]):
        for col in range(classified_image.shape[1]):
            vv_val = float(vv_data[row][col])
            vh_val = float(vh_data[row][col])
            r_val = float(r_data[row][col])
            water = water_thresh.is_member(vv_val, vh_val, r_val)
            flooded = flooded_thresh.is_member(vv_val, vh_val, r_val)
            not_flooded = not_flooded_thresh.is_member(vv_val, vh_val, r_val)
            if water:
                if vv_cov[row][col] > w_vv_cov_thresh:
                    classified_image[row][col] = 1
                else:
                    classified_image[row][col] = 0
            elif flooded:
                if vv_cov[row][col] > f_vv_cov_thresh or vh_cov[row][col] > f_vh_cov_thresh:
                    classified_image[row][col] = 4
                else:
                    classified_image[row][col] = 3
            else:
                if vv_cov[row][col] > nf_vv_cov_thresh or vh_cov[row][col] > nf_vh_cov_thresh:
                    classified_image[row][col] = 4
                else:
                    classified_image[row][col] = 2
    return(classified_image)

###############################################################################

def refined_multilook_classify(filtered_date_sets, 
                               min_inundation, 
                               ow_min_count=False,
                               ni_min_count=False,
                               iv_min_count=False,
                               return_counts=False
                              ):
    ''' Function to perform a refined classification integrating the
    results of the multilooked classifications.  The final refined
    classification is based off the count of occurences of each class
    in each pixel across the multilooked classifications.
    Parameters:
    - filtered_date_sets: List of DateSets that have multilooked classifications
    - min_inundation: Minimum inundation state classification
    - ow_min_count: Minimum count threshold for permanent open water
    - ni_min_count: Minimum count threshold for permanent not inundated
    - iv_min_count: Minimum count threshold for permanent inundated vegetation
    - return_counts: Option to return the occurence counts for each class
    '''
    # If minimum count thresholds have not been provided, set them to
    # values scaled off the number of dates used
    ndates = len(filtered_date_sets)
    if not ow_min_count: ow_min_count = ndates/2
    if not ni_min_count: ni_min_count = ndates
    if not iv_min_count: iv_min_count = ndates/2
    # Count number of occurences of each class in each pixel across
    # classified multi-looked dates
    nrows = np.size(min_inundation, 0)
    ncols = np.size(min_inundation, 1)
    ow_count = np.zeros((nrows, ncols))
    iv_count = np.zeros((nrows, ncols))
    ni_count = np.zeros((nrows, ncols))
    for ds in filtered_date_sets:
        for row in range(nrows):
            for col in range(ncols):
                val = ds.class_3x3[row][col]
                if val == 1:
                    ow_count[row][col] += 1
                elif val == 2:
                    ni_count[row][col] += 1
                elif val == 3:
                    iv_count[row][col] += 1
    # Make refined classification based off class occurence counts
    refined_inundation = copy.copy(min_inundation)
    for ds in filtered_date_sets:
        for row in range(nrows):
            for col in range(ncols):
                if ow_count[row][col] >= ow_min_count:
                    refined_inundation[row][col] = 1
                elif ni_count[row][col] >= ni_min_count:
                    refined_inundation[row][col] = 2
                elif iv_count[row][col] >= iv_min_count:
                    refined_inundation[row][col] = 3
                else:
                    refined_inundation[row][col] = 4
    # Return the class occurrence counts, if desired
    if return_counts:
        counts = [ow_count, iv_count, ni_count]
        return([refined_inundation, counts])
    else:
        return(refined_inundation)

###############################################################################

def prompt_already_multilooked(pn=pn):
    '''Asks the user if they have already classified and exported the
    multi-looked images and loads them if so.  If not, starts workflow to
    create them.
    
    Parameters:
    pn: Python version number (2 or 3)
    '''
    question = ("Have you already classified and exported the " +
               "multi-looked images? (y/n)")
    while True:
        try:
            if pn == 2:
                user_in = raw_input(question)
            elif pn == 3:
                user_in = input(question)
        except:
            print("Invalid response, try again")
            continue
        else:
            affirmative_answers = ["y", "Y","yes", "Yes"]
            negative_answers = ["n", "N", "No", "no"]
            if user_in in affirmative_answers:
                load_classified_3x3s(filtered_date_sets, out_directory)
                load_multilooked(filtered_date_sets, multi_directory)
                print("Loaded previously classified multi-looked images.")
                break
            elif user_in in negative_answers:
                multilook_all_dates(filtered_date_sets,
                            water_thresh, 
                            flooded_thresh,
                            multi_directory,
                            ulx, uly, lrx, lry, proj
                                   )
                print("Multi-looking and classification complete.")
                break
            else:
                print("Invalid response, try again")
                continue  
        
###############################################################################

def multilook_classify(date_set, 
                       water_thresh, 
                       flooded_thresh
                      ):
    '''Classifies a multi-looked image and returns the result.
    
    Parameters:
    - date_set: A DateSet object from date to be classified
    - water_thresh: ClassThreshold object for open water class
    - flooded_thresh: ClassThreshold object for inundated class
    '''
    vv_3x3 = moving_window(date_set.vv_df.read_data())
    vh_3x3 = moving_window(date_set.vh_df.read_data())
    r_3x3 = vv_3x3/vh_3x3
    class_3x3 = triple_classify(vv_3x3, vh_3x3, r_3x3, 
                                water_thresh, 
                                flooded_thresh 
                               )
    datef = date_set.datef
    date_set.close()
    return(class_3x3, datef)

###############################################################################

def multilook_all_dates(filtered_date_sets,
                        water_thresh, 
                        flooded_thresh,
                        multi_directory,
                        ulx, uly, lrx, lry, proj,
                        classify=True
                        ):
    '''Performs multi-looking on all input dates and stores result as an 
    attribute (class_3x3) in the input DateSets.
    
    Parameters:
    - filtered_date_sets: List of filtered DateSets
    - water_thresh: ClassThreshold object for open water class
    - flooded_thresh: ClassThreshold object for inundated class
    - multi_directory: Directory where multilooked images are stored
    - ulx, uly, lrx, lry: Corner coordinates for exporting
    - proj: Projection for exporting
    '''
    nds = len(filtered_date_sets)
    for idx, ds in enumerate(filtered_date_sets):
        print(
            str(datetime.now().time().strftime("%H:%M:%S")) 
            + ") Processing data from " 
            + ds.datef 
            + "... [" 
            + str(idx+1) 
            + "/" 
            + str(nds) 
            + "]"
        )
        vv_3x3 = moving_window(ds.vv_df.read_data())
        vv_out = multi_directory + ds.vv_df.name.split('.')[0] + '3x3.tif'
        proc_export(vv_3x3, ulx, uly, lrx, lry, vv_out, proj, True)
        vh_3x3 = moving_window(ds.vh_df.read_data())
        vh_out = multi_directory + ds.vh_df.name.split('.')[0] + '3x3.tif'
        proc_export(vh_3x3, ulx, uly, lrx, lry, vh_out, proj, True)
        r_3x3 = vv_3x3/vh_3x3
        if classify:
            ds.class_3x3 = triple_classify(
                vv_3x3, vh_3x3, r_3x3, 
                water_thresh, flooded_thresh
            )
        ds.close()
        
###############################################################################

def interactive_area_plot(classified_date_sets):
    '''Creates an interactive time series plot showing the total area and 
    percentage of total area inundated.
    
    Parameters:
    - classified_date_sets: List of DateSets with classified results
    '''
    # Modified code from https://plot.ly/python/range-slider/
    dateps = []
    areas= []
    aps = []
    for ds in classified_date_sets:
        dateps.append(ds.datep)
        [ds.area, ds.percent, total] = calculate_area_inundated(ds.class_3x3)
        areas.append(0.0001*ds.area)
        aps.append([0.0001*ds.area, ds.percent])
    area_trace = go.Scatter(
      x = dateps,
      y = areas,
      name = "Inundated area",
      text = [str(ap[0]) + "ha (%.2f%%)" % ap[1] for ap in aps],
      yaxis = "y"
    )

    data = [area_trace]

    # style all the traces
    for k in range(len(data)):
        data[k].update(
            {
                "hoverinfo": "name+x+text",
                "line": {"width": 0.5}, 
                "marker": {"size": 8},
                "mode": "lines+markers",
                "showlegend": False
            }
        )

    layout = {
      "dragmode": "zoom", 
      "hovermode": "x", 
      "legend": {"traceorder": "reversed"}, 
      "margin": {
        "t": 100, 
        "b": 100
      }, 
      "xaxis": {
        "autorange": True, 
        "range": ["2017-10-31 18:36:37.3129", "2018-05-10 05:23:22.6871"], 
        "rangeslider": {
          "autorange": True, 
          "range": ["2017-10-31 18:36:37.3129", "2018-05-10 05:23:22.6871"]
        }, 
        "type": "date"
      }, 
      "yaxis": {
        "anchor": "x", 
        "autorange": True, 
        "domain": [0, 1], 
        "linecolor": "#673ab7", 
        "mirror": True, 
        "range": [0,10000], 
        "showline": True, 
        "side": "right", 
        "tickfont": {"color": "#673ab7"}, 
        "tickmode": "auto", 
        "ticks": "",
        "title": "Inundated area (ha)",
        "titlefont": {"color": "#673ab7"}, 
        "type": "linear", 
        "zeroline": False
        }
    }
    plotly.offline.init_notebook_mode(connected=True)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)
        
###############################################################################
        
def min_max_inundation(typical_inundation, classified_date_sets):
    '''Returns the minimum and maximum inundation state by comparing all
    classified dates.
    
    Parameters:
    - typical_inundation: Classification product from the multi-temporal 
        averages
    - classified_date_sets: DateSets that contain classifications for the 
        multi-looked products
    '''
    max_inundation = copy.copy(typical_inundation)
    min_inundation = copy.copy(typical_inundation)
    for row in range(typical_inundation.shape[0]):
        for col in range(typical_inundation.shape[1]):
            pixel_class_values = [
                int(ds.class_3x3[row][col]) for ds in classified_date_sets
            ]
            default_class = typical_inundation[row][col]
            if 3 in pixel_class_values: max_inundation[row][col] = 3
            
            if 2 in pixel_class_values: min_inundation[row][col] = 2
            elif 1 in pixel_class_values: min_inundation[row][col] = 1
            else: min_inundation[row][col] = 3                
    return(min_inundation, max_inundation) 

###############################################################################
        
def classified_plot(classified_image, datef=False):
    '''Plots a classified product using 5 classes.
    
    Parameters:
    - classified_image: 2D array of classified values
    - datef (optional): Formatted date string to be plotted along y-axis
    '''
    fig, ax = plt.subplots(figsize=(9.5,9.5))
    colors = ['b', 'c', 'g', 'y', 'm']
    cmap = matplotlib.colors.ListedColormap(colors)
    cax = ax.imshow(classified_image, cmap=cmap, vmin = -.2, vmax = 4.2)
    cbar = fig.colorbar(cax, ticks=[0,1,2,3,4])
    cbar.ax.set_yticklabels(
        ['Permanent Open Water', 'Seasonal Open Water', 
         'Permanent Not Inundated', 'Permanent Inundated Vegetation', 
         'Seasonal Inundation']
    )
    title_str = "Classified using VV, VH, and VV/VH brightness values: "
    if datef: title_str += datef
    else: title_str += "Multi-temporal average"
    ax.set_title(title_str)
    
###############################################################################
    
def classified_plot2(classified_image, datef=False, colors=['b','g','y']):
    '''Plots classified product with 3 classes (Open water, not inundated,
    and inundated).
    
    Parameters:
    - classified_image: 2D array of classified values
    - datef (optional): Formatted date string to be plotted along y-axis
    - colors: List containing the names of the colors for the 3 classes
    '''
    fig, ax = plt.subplots(figsize=(9.5,9.5))
    cax = ax.imshow(
        classified_image, 
        cmap=matplotlib.colors.ListedColormap(colors), 
        vmin = 0, vmax = 4
    )
    cbar = fig.colorbar(cax, ticks=[.67,2,3.33])
    cbar.ax.set_yticklabels(['Open Water', 'Not Inundated', 'Inundated'])
    title_str = "Classified using VV, VH, and VV/VH brightness values: "
    if datef: title_str += datef
    else: title_str += "Multi-temporal average"
    ax.set_title(title_str)
    
###############################################################################
    
def refined_plot(classified_image, datef=False, colors=['b', 'w', 'y', 'g']):
    '''Plots the results of the refined classification (4 classes).
    
    Parameters:
    - classified_image: 2D array of refined classification values
    - datef (optional): Formatted date string to be plotted along y-axis
    '''
    fig, ax = plt.subplots(figsize=(9.5,9.5))
    cax = ax.imshow(
        classified_image, 
        cmap=matplotlib.colors.ListedColormap(colors), 
        vmin = .5, vmax = 4.5
    )
    cbar = fig.colorbar(cax, ticks=[1,2,3,4])
    cbar.ax.set_yticklabels(
        ['Permanent open water', 
         'Not inundated', 
         'Permanent inundated vegetation', 'Seasonal inundation'
        ]
    )
    title_str = "Refined classification using VV, VH, and VV/VH values: "
    if datef: title_str += datef
    else: title_str += "Multi-temporal average"
    ax.set_title(title_str)    
    
###############################################################################
    
def extract_spatial_metadata(data_file):
    '''Extracts and returns the projection, upper left, and lower right 
    corners' projected coordinates to use when exporting.
    
    Parameters:
    - data_file: DataFile
    '''
    data_file.read_data()
    prof = data_file.raw.profile
    affine = prof['transform']
    ulx, uly = affine[2], affine[5]
    height = prof['height']
    width = prof['width']
    dx = affine[0]
    dy = affine[4]
    lrx = ulx + width*dx
    lry = uly + height*dy
    proj = "EPSG:" + str(prof['crs']).split('EPSG:')[-1]
    data_file.close()
    return(ulx, uly, lrx, lry, proj)

###############################################################################

def proc_export(array, 
                ulx, uly, lrx, lry, 
                outpath, 
                proj="EPSG:32605", 
                clean_temp=True
               ):
    '''Exports array as a raster file (.tif).
    
    Parameters: 
    - array: 2D image array
    - ulx, uly: Projected x and y coordinates of upper left corner
    - lrx, lry: Projected x and y coordinates of lower right corner
    - outpath: Path for output file
    - proj: Projection in GDAL-readable format
    '''
    if not outpath.endswith(".tif"):
        print("Output filename must end with .tif")
        return(False)
    else:
        # Create temporary TIF file
        temp_path = outpath.split(".tif")[0] + "_temp.tif"
        temp_image = Image.fromarray(array)
        temp_image.save(temp_path)
        
        # Add spatial information with GDAL
        translate_command = "gdal_translate"
        translate_command += " -a_srs " + proj
        translate_command += " -a_ullr " + str(ulx) + " " + str(uly)
        translate_command += " " + str(lrx) + " " + str(lry)
        translate_command += " " + temp_path + " " + outpath
        subprocess.call(translate_command, shell=True)
        
        # Clean temporary file
        if clean_temp:
            clean_command = "rm " + temp_path
            subprocess.call(clean_command, shell=True)
###############################################################################

def export_classified_3x3(classified_date_sets, out_dir):
    '''Exports the results of the classification of the multi-looked images as 
    .tif files.
    
    Parameters:
    - classified_date_sets: List of DateSets with multi-look classifications
    - out_dir: Output directory
    '''
    if out_dir[-1] != '/': out_dir += '/'
    for ds in classified_date_sets:
        c3x3 = ds.class_3x3
        outpath = out_dir + ds.date + '_inundation.tif'
        proc_export(c3x3, ulx, uly, lrx, lry, outpath, proj, True)
###############################################################################

def load_multilooked(filtered_dates_sets, multi_directory):
    for ds in filtered_date_sets:
        vv_out = multi_directory + ds.vv_df.name.split('.')[0] + '3x3.tif'
        vv_raw = rasterio.open(vv_out)
        ds.vv_3x3 = vv_raw.read(
            1, out_shape=(1, int(vv_raw.height), int(vv_raw.width))
        )
        vh_out = multi_directory + ds.vh_df.name.split('.')[0] + '3x3.tif'
        vh_raw = rasterio.open(vh_out)
        ds.vh_3x3 = vh_raw.read(
        1, out_shape=(1, int(vh_raw.height), int(vh_raw.width))
        )
        ds.r_3x3 = ds.vv_3x3/ds.vh_3x3

        
###############################################################################

def load_classified_3x3s(filtered_dates_set, out_dir):
    '''Loads the previously exported multi-looked classifications and appends
    them to existing DateSets.
    
    Parameters:
    - filtered_date_sets: DateSets that have been filtered for inclusion
    - out_dir: Path to directory where previously exported results are stored
    '''
    for ds in filtered_date_sets:
        data_dir_search = out_dir + "*" + ds.date + "_inundation.tif"
        search = glob(data_dir_search)
        if len(search) == 1:
            raw = rasterio.open(search[0])
            ds.class_3x3 = raw.read(
                1, out_shape=(1, int(raw.height), int(raw.width))
            )        
            
###############################################################################

def calculate_area_inundated(classified, res=10):
    '''Calculates and returns area inundated (and percent area inundated).
    
    Parameters:
    - classified: Classified 2D array
    - res: Pixel size (in projected units, e.g. meters)
    '''
    nrow, ncol = classified.shape
    xlength = ncol*res
    ylength = nrow*res
    total_area = xlength * ylength
    count_inundated = 0
    res2 = res**2
    for row in range(nrow):
        for col in range(ncol):
            if classified[row][col] == 3: count_inundated += 1
    area_inundated = count_inundated*res2
    percent_inundated = float(area_inundated)*100/total_area
    return([area_inundated, percent_inundated, total_area])       

###############################################################################
            
def proc_resample(path, overwrite=False):
    '''Resamples a file using the GDAL shell library and creates a new file.
    
    Parameters:
    - path: Path to the file to be resampled
    - overwrite: Option to overwrite existing resampled result
    '''
    output = path.split('.tif')[0] + '_resampled.tif'
    pieces = ["gdalwarp -tr 100 100", path, output]
    if overwrite: pieces.append("-overwrite")
    gdalwarp_command = " ".join(pieces)
    print(gdalwarp_command)
    subprocess.call(gdalwarp_command, shell=True)

def resample_all(directory, overwrite=False):
    '''Resamples all files in a specified directory.
    
    Parameters:
    - directory: Path to directory
    - overwrite: Option to overwrite existing resampled output
    '''
    figs = os.listdir(directory)
    paths = [os.path.join(directory, fig) for fig in figs] 
    for path in paths:
        if 'resampled' not in path and path.endswith('.tif'):
            proc_resample(path, overwrite) 
            
###############################################################################
        
def proc_gdalwarp(infile,  proj="EPSG:32605", outfile="null"):
    '''Projects a raster into specified projection.
    
    Parameters:
    - infile: Path to file to be projected
    - proj: String in GDAL-readable format specifying projection (default is
        WGS 84 - UTM Zone 5N).
    - outfile: Path to output file (filename will be automatically generated
        if none is specified)
    '''
    if outfile == "null":
        ext = infile.split(".")[-1]
        outfile = infile.split(".")[0] + "_proj." + ext
    warp_command = "gdalwarp -t_srs " + proj + " " + infile + " " + outfile
    subprocess.call(warp_command, shell=True)
    
###############################################################################

def proc_translate(infile, outfile="null"):
    '''Converts a raster file into ENVI format and produces an ENVI header 
    file while doing so.
    
    Parameters:
    - infile: Path to file to be converted
    - outfile: Path to output file
    '''
    if outfile == "null":
        outfile = infile.split(".")[0] + ".img"
    translate_command = "gdal_translate -of ENVI " + infile + " " + outfile
    subprocess.call(translate_command, shell=True)
    
###############################################################################

<br>
<hr>
<br>
<font face="Century Gothic">
<font size="3"><b>Load Data</b><br></font>
For this exercise, we will be using a VV/VH Sentinel-1 data stack over Selawik, Alaska. The town of Selawik is located in the northwest of Alaska on the coast to the Chukchi and Bering sea. It is prone to heavy rains and extensive inundation during the breakup and summer seasons. We will use Sentinel-1 data to map inundated vegetation and understand the persistence of inundation in the area.
    
Before we get started, let's first <b>create a working directory for this analysis and download the relevant data into the notebook"</b>
</font>

In [ ]:
project_dir = "/home/jovyan/notebooks/SAR_Training/English/Ecosystems/S1-InundationMapping/"
time_series_path = 's3://asf-jupyter-data/S1-InundationMapping.zip'
time_series = os.path.basename(time_series_path)
!aws --region=us-east-1 --no-sign-request s3 cp $time_series_path $time_series

In [ ]:
if asfn.path_exists(time_series):
    asfn.asf_unzip(os.getcwd(), time_series)
    os.remove(time_series)
os.chdir(project_dir)

In [ ]:
site_name = "SelawikZoom"
data_directory = "./data/" + site_name + "/"
out_directory = "./out/" + site_name + "/"
multi_directory = data_directory + "multi/"

<font face="Century Gothic">
<font size="3"><b>Selection Process</b><br></font>
After selecting the desired time range, this section outputs a list of the files, their VV, VH, and VV/VH ratio value <br>ranges, image thumbnails, and plots of image brightness.  High increases in image brightness may be due to <br>different reasons such as the presence of snow or ice cover, wind and weather changes, or calibration errors, but <br>with the selection options below, users can visually evaluate the image collection and select which dates to <br>include or exclude from analysis.
</font>

In [ ]:
# Get list of files in data_directory with the correct file extension
# and contain the search string in their filename
ext = ".tif"
search = "subset"
data_files = get_files(data_directory, ext, search, "new")

# Find dates
dates = find_dates(data_files)

# Make DateSets (store VV and VH DataFiles into a single object)
date_sets = make_datesets(data_files, dates)
datesf = [ds.datef for ds in date_sets]

# Find minimum and maximum VV and VH values of the DataFiles (use these to 
# standardize color limits)
[[vv_min, vv_max], [vh_min, vh_max], [r_min, r_max]] = find_minmax(date_sets)

# Print dates
print("\nDates:")
datesf

In [ ]:
#### Make adjustments to plotting color limits
vv_min = 0 #minimum VV value
vv_max = .15 #maximum VV value
vh_min = 0#minimum VH value
vh_max = .03 #maximum VH value
r_min = 0 #minimum VV/VH value
r_max = 15 #maximum VV/VH value

In [ ]:
###############################################################################
# Plot all images with checkboxes to select images for exclusion:
ws3 = multi_checkbox_widget_dateimages(
    date_sets, 
    vv_min, vv_max, 
    vh_min, vh_max,
    r_min, r_max
)
ws3

In [ ]:
# Get indices of checked options
#(Can only be run once per creation of the box above, need to re-run code above)
excluded_dates3 = get_excluded_dates3(ws3, date_sets)
print("Dates to be excluded:")
excluded_dates3

In [ ]:
###############################################################################
# Interactive plot time series of average brightness per image
interactive_backscatter_plot(date_sets)

In [ ]:
# Make time slider to select start and end dates
selection_range_slider = make_timeslider(date_sets)
selection_range_slider

In [ ]:
# Get start and end dates from time slider (re-run this to get updated times)
slider_min, slider_max = get_slider_vals(selection_range_slider)
# Filter dates according to checkbox and time slider
filtered_date_sets = filter_date_sets(date_sets, excluded_dates3, slider_min, slider_max)

# Extract spatial metdata from filtered images to export classified products
ulx, uly, lrx, lry, proj = extract_spatial_metadata(filtered_date_sets[0].vv_df)

<font face="Century Gothic">
<font size="4"><b>Classification Overview</b></font><br>
Typical inundation is defined as the classification of the multi-temporal average iamges (Ia).<br>
<br>
<font size="3"><b>Classify Inundation:</b></font>
<ul>
    <li> Calculate multi-temporal average for VV, VH, and VV/VH </li>
<li> Derive typical classification from multi-temporal averages with rules-based classification</li>
<li>For each date: </li>
    <ul>
        <li> Calculate multi-looked view for VV, VH, and VV/VH </li>
        <li>Perform initial classification</li>
        <li>Compare to corresponding multi-temporal average to determine changes</li>
        <li>Make refined classification based on change with rules</li>
    </ul>
    <li>For each pixel:</li><ul>
    <li>Loop over all refined classifications to find minimum and maximum inundation</li></ul>
    </ul>
    </font>

<font face="Century Gothic">
<font size="3"><b>
Multi-Temporal Averages <br></font></b>
This section produces a multi-temporal radar backscatter image (Ia) by averaging the data from each acquisition of<br>
VV, VH, and VV/VH over the selected date ranges.<br>
<br>
In general, SAR data can be noisy and speckled.  However, averaging images reduces speckle and smooths the imagery<br>
and can be used to examine changes over time.  In areas where land cover and terrain remain the same, the level of<br>
speckle visible in individual scenes is reduced, while dynamic areas of environmental change will reflect<br>
variations in backscatter.  This can be particularly evident in areas of inundation, as open water and inundated<br>
vegetation are at opposite ends of the range of radar backscatter values.
</font>

In [ ]:
# Calculate multi-temporal average for VV, VH, and VV/VH over selected date range
(vv_avg, vh_avg, r_avg, vv_cov, vh_cov) = calculate_multitemporal_avg(filtered_date_sets)
# Plot them:
fig_xsize = 10
fig_ysize = 7
f = vv_vh_r_plot(vv_avg, vh_avg, r_avg, 
                 vv_min, vv_max, 
                 vh_min, vh_max, 
                 r_min, r_max, 
                 "Multi-temporal averages", 
                 fig_xsize, fig_ysize
                )


In [ ]:
# Large plot of multi-temporal average of VV values to inspect pixel values
fig_xsize = 8
fig_ysize = 8
big_fig(fig_xsize, fig_ysize)
gray_plot(vv_avg, vv_min, vv_max)

In [ ]:
# Full-resolution plot of multi-temporal average of VV 
dpi = 192 #change this to match your monitor's DPI
cursor = True #option to enable data cursor in resulting image
full = True #option to increase Jupyter Notebook width to monitor size 
#pixel2pixel_plot(vv_avg, vv_min, vv_max, dpi, cursor, full) # Comment/uncomment this line to run

In [ ]:
# Large plot of multi-temporal average of VH values to inspect pixel values
big_fig(fig_xsize, fig_ysize)
gray_plot(vh_avg, vh_min, vh_max)

In [ ]:
# Full-resolution plot of multi-temporal average of VH
dpi = 192 #change this to match your monitor's DPI
cursor = True #option to enable data cursor in resulting image
full = True #option to increase Jupyter Notebook width to monitor size 
# pixel2pixel_plot(vh_avg, vh_min, vh_max, dpi, cursor, full) # Comment/uncomment this line to run

In [ ]:
# Large plot of multi-temporal average of VV/VH values to inspect pixel values
big_fig(fig_xsize, fig_ysize)
gray_plot(r_avg, r_min, r_max)

In [ ]:
# Full-resolution plot of multi-temporal average of VV/VH
dpi = 192 #change this to match your monitor's DPI
cursor = True #option to enable data cursor in resulting image
full = True #option to increase Jupyter Notebook width to monitor size 
# pixel2pixel_plot(r_avg, r_min, r_max, dpi, cursor, full) # uncomment this line to run

In [ ]:
# Plot coefficient of variation for VV and VH
f = vv_vh_cov_plot(vv_cov, vh_cov,
                 0, 1, 
                 0, 1, 
                 "Coefficient of Variation", 
                 10, 7
                )

In [ ]:
# Large plot of VV coefficient of variation
big_fig(fig_xsize, fig_ysize)
gray_plot(vv_cov, 0, 1)

In [ ]:
# Full resolution plot of VV/VH coefficient of variation
dpi = 192 #change this to match your monitor's DPI
cursor = True #option to enable data cursor in resulting image
full = True #option to increase Jupyter Notebook width to monitor size 
# pixel2pixel_plot(r_cov, 0, 1, dpi, cursor, full) # Comment/uncomment this line to run

In [ ]:
# Calculate VV/VH coefficient of variation
r_cov = calculate_r_cov(filtered_date_sets)
# Large plot of VV/VH coefficient of variation
big_fig(fig_xsize, fig_ysize)
gray_plot(r_cov, 0, 1)

In [ ]:
# Full resolution plot of VV/VH coefficient of variation
dpi = 192 #change this to match your monitor's DPI
cursor = True #option to enable data cursor in resulting image
full = True #option to increase Jupyter Notebook width to monitor size 
# pixel2pixel_plot(r_cov, 0, 1, dpi, cursor, full) # Comment/uncomment this line to run

In [ ]:
# Export multi-temporal averages and coefficient of variation
# VV
vv_avg_out = out_directory + 'vv_avg.tif'
proc_export(vv_avg, ulx, uly, lrx, lry, vv_avg_out, proj, True)
# VH
vh_avg_out = out_directory + 'vh_avg.tif'
proc_export(vh_avg, ulx, uly, lrx, lry, vh_avg_out, proj, True)
# VV/VH
r_avg_out = out_directory + 'r_avg.tif'
proc_export(r_avg, ulx, uly, lrx, lry, r_avg_out, proj, True)
# VV CoV
vv_cov_out = out_directory + 'vv_cov.tif'
proc_export(vv_cov, ulx, uly, lrx, lry, vv_cov_out, proj, True)
# VH CoV
vh_cov_out = out_directory  + 'vh_cov.tif'
proc_export(vh_cov, ulx, uly, lrx, lry, vh_cov_out, proj, False)

In [ ]:
###############################################################################

# Derive typical inundation from multi-temporal averages with rules-based 
# classification. These thresholds are determined through pixel inspection 
# and analysis from temporal averages, and can be adjusted by the user. 

# Set threshold values for the different classes
'''Example:
class_thresh = ClassThresholds(class_vv_min, class_vv_max,
                                class_vh_min, class_vh_max,
                                class_r_min, class_r_max
                                )
'''
water_thresh = ClassThresholds(0, 0.07,
                               0, 0.0045,
                               0, 1000
                              )

flooded_thresh = ClassThresholds(0.12, 1,
                                 .0045, 1.5,
                                 4, 50
                                )

# Typical inundation, or the average inundation over a time period, is 
# found using a triple classification function. In this triple classification,
# the function runs through and uses each of the different class thresholds
# from VV, VH,and VV/VH to create a single classification output. 

# Find typical inundation from multi-temporal averages of VV, VH, and VV/VH
typical_inundation = triple_classify(
    vv_avg, vh_avg, r_avg, 
    water_thresh, flooded_thresh)

In [ ]:
# Plot typical inundation state
classified_plot2(typical_inundation)
#plt.savefig('typical_inundation.pdf') #Save figure as pdf

In [ ]:
# Export typical inundation state as GeoTIFF
typical_out = out_directory + 'typical_inundation.tif'
proc_export(typical_inundation, ulx, uly, lrx, lry, typical_out, proj, True)

<font face="Century Gothic">
<font size="3"><b>
Multi-looked Classification
    </font></b><br>
Inundation extent can fluctuate greatly.  Individual images or dates can also be classified to analyze temporal variability and make specific date comparisons.  Multi-looked views are applied to each date's VV, VH, and VV/VH images to help smooth and reduce speckle.  The multi-look view uses a 3x3 moving window to average neighboring pixels. <br>
<br><b>For each date: </b><br>
<ul>
    <li> Calculate multi-looked view for VV, VH, and VV/VH</li>
    <li> Perform rules-based classification</li>
</ul>
    
Note: I precalculated all necessary multilooked images. <font color='rgba(200,0,0,0.2)'><b>So please answer the question in the next code cell with YES!</b></font>
</font>

In [ ]:
# Ask user if they have previously classified and exported the multi-looked images.
# If the user has not classified and exported the multi-looked images, 
    # the script will multi-look all filtered dates using a 3x3 moving window, 
    # perform rules-based classification,
    # and save the classification to the DateSet objects in filtered_date_sets.
    # WARNING: This takes several minutes per date!
# If the user has already classified and exported the multi-looked images, 
    # the script will load the exported .tif files located in ./out/
    # and append them to the DateSet objects in filtered_date_sets.
    
# NOTE: This step takes very long!! Therefore, I have preprocessed all multi-looked data for you. 
# PLEASE ANSWER THE PROMPTED QUESTION WITH YES for this data set!!
prompt_already_multilooked()

In [ ]:
# Export all classified multi-looked dates
export_classified_3x3(filtered_date_sets, out_directory)

In [ ]:
# Calculate and plot change in inundated area over time
interactive_area_plot(filtered_date_sets)

In [ ]:
# Find minimum and maximum inundation
min_inundation, max_inundation = min_max_inundation(
    typical_inundation, 
    filtered_date_sets
)

In [ ]:
# Plot minimum inundation
classified_plot2(min_inundation, "Minimum inundation")
#plt.savefig('minimum_inundation.pdf') #Save figure as pdf


In [ ]:
# Plot maximum inundation
classified_plot2(max_inundation, "Maximum inundation")
#plt.savefig('maximum_inundation.pdf') #Save figure as pdf


In [ ]:
# Export minimum and maximum inundation maps as GeoTIFF
min_out = out_directory + 'minimum_inundation.tif'
max_out = out_directory + 'maximum_inundation.tif'
proc_export(min_inundation, ulx, uly, lrx, lry, min_out, proj, True)
proc_export(max_inundation, ulx, uly, lrx, lry, max_out, proj, True)


<font face="Century Gothic">
<font size="3"><b>
Multi-looked-based refined classification
    </font></b><br>
Detect areas of seasonal inundation using change detected from multi-looked dates. <br>

</font>

In [ ]:
# Perform refined classification based on number of class occurrences per pixel
# across all multi-looked dates.  User can elect to alter the default minimum
# counts below by changing the min_count values to a number less than 
# or equal to the number of multi-looked dates.

# Class occurrence count thresholds
# (Leave as False to use default values, or change to numeric values)
ow_min_count=False #Open water minimum count
ni_min_count=10 #Not inundated minimum count
iv_min_count=10 #Inundated vegetation minimum count

refined_inundation = refined_multilook_classify(filtered_date_sets, min_inundation,
                                               ow_min_count, ni_min_count, iv_min_count
                                               )

In [ ]:
# Plot refined classification
refined_plot(refined_inundation, "Refined classification")

In [ ]:
# Export refined classification as GeoTIFF
refined_out = out_directory + "refined_classification2_dswe.tif"
proc_export(refined_inundation, ulx, uly, lrx, lry, refined_out, proj, True)

<hr>
<br>
<font face="Century Gothic" size="5"> <b> 8. Conclusion</b> </font>

<font face="Century Gothic" size="3">Multi-temporal SAR data data from Sentinel-1 are a good basis for identifying inundated areas and distinguish seasonal inundation from short term inundation. Note, however, that due to limited penetration into dense vegetation, the performance of C-band Sentinel-1 data for inundation mapping in Colombia will be limited. A better choice will be L-band SAR data from future missions such as NISAR. The higher penetration of L-band will improve inundation mapping performance. The same workflow can be used for these future L-band data. 

For a bit more information on change detection and SAR in general, please look at the recently published <a href="https://gis1.servirglobal.net/TrainingMaterials/SAR/SARHB_FullRes.pdf" target="_blank">SAR Handbook: Comprehensive Methodologies for Forest Monitoring and Biomass Estimation</a>.
</font> 
<hr>
<br>

<font face="Calibri" size="2"> <i>Exercise7-InundationMappingfromSARTimeSeries-Example.ipynb - Version 1.3.0 - April 2021
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>namespace asf_notebook</li>
    </ul>
    </i>
</font>